# MPS basics

In this notebook, we introduce the `SimpleMPS` class from `toycodes/a_mps.py` and the model class from `toycodes/b_model.py`.

In [1]:
# standard imports and cosmetics

import numpy as np

import matplotlib.pyplot as plt

np.set_printoptions(precision=5, suppress=True, linewidth=100, threshold=50)
plt.rcParams['figure.dpi'] = 150

## SimpleMPS class from `toycodes/a_mps.py`

The file `toycodes/a_mps.py` defines a `SimpleMPS` class, that provides methods for expectation values and the entanglement entropy. 

You can initialize an inital product state MPS with the provided functions
- `init_FM_MPS` to initialize the state $\lvert \uparrow \uparrow \cdots \uparrow \uparrow \rangle$, and
- `init_Neel_MPS` to initialize the Neel state $\lvert \uparrow \downarrow \cdots \uparrow \downarrow \rangle$

In [2]:
import toycodes.a_mps
from toycodes.a_mps import SimpleMPS, init_FM_MPS, init_Neel_MPS

In [3]:
psi_FM = init_FM_MPS(L=10)
print(psi_FM)

In [4]:
Z = np.diag([1., -1.])

print(psi_FM.site_expectation_value(Z))

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


### Exercise: expectation values and entropy

- Initialize a Neel state MPS. Print the expectation values of `Z`
- Print the entanglement entropy. What do you expect? Why do you get so many numbers, and not just one?
  *Tip*: read the code ;-)
- Extract the half-chain entanglement entropy, i.e., the entropy when cutting the chain into two equal-length halves.

In [5]:
psi_Neel = init_Neel_MPS(L=10)
print(psi_Neel.site_expectation_value(Z))

[ 1. -1.  1. -1.  1. -1.  1. -1.  1. -1.]


In [6]:
print(psi_Neel.entanglement_entropy()) 
# one value for cutting at each bond!

[-0. -0. -0. -0. -0. -0. -0. -0. -0.]


In [7]:
print("half-chain entropy: ", psi_Neel.entanglement_entropy()[(psi_Neel.L - 1)//2])
# a product state has no entanglement!

half-chain entropy:  -0.0


### Exercise: `init_PM_MPS()`

Write a function `init_PM_MPS` to initialize the state $\lvert \rightarrow \rightarrow \cdots \rightarrow \rightarrow \rangle$,
where $\lvert  \rightarrow \rangle = \frac{1}{\sqrt{2}} \big( \lvert\uparrow \rangle + \lvert\downarrow\rangle \big) $ is the spin-1/2 state pointing in plus x direction.

*Tip*: the code should be similar to `init_FM_MPS`.

In [8]:
def init_PM_MPS(L, bc='finite'):
    """Return a paramagnetic MPS (= product state with all spins pointing in +x direction)"""
    d = 2
    B = np.zeros([1, d, 1], dtype=float)
    B[0, 0, 0] = 1./np.sqrt(2)
    B[0, 1, 0] = 1./np.sqrt(2)
    S = np.ones([1], dtype=float)
    Bs = [B.copy() for i in range(L)]
    Ss = [S.copy() for i in range(L)]
    return SimpleMPS(Bs, Ss, bc=bc)

In [9]:
psi_PM = init_PM_MPS(L=10)

## Model class

The file `toycodes/b_model.py` defines a `TFIModel` class representing the transverse field Ising model        
$$H = - J \sum_{i} Z_i Z_{i+1} - g \sum_{i} X_i$$

It provides the Hamiltonian both in the form of bond-terms `H_bonds` (as required for TEBD) and in the form of an MPO `H_mpo` (as required for DMRG).
You can use `H_bonds` with `SimpleMPS.bond_expectation_values` to evalue the energy:


In [10]:
from toycodes.b_model import TFIModel

L = 10
J = 1.
g = 1.2
model = TFIModel(L=L, J=J, g=g, bc='finite')

print("<H_bonds> = ", psi_FM.bond_expectation_value(model.H_bonds))
print("energy:", np.sum(psi_FM.bond_expectation_value(model.H_bonds)))
# (make sure the model and state have the same length and boundary conditions!)

print("should be", (L-1)* (-J) * (1. * 1.) + L * (-g) * (0.) )

<H_bonds> =  [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
energy: -9.0
should be -9.0


### Exercise

- Find the code where the MPO `W` for this model is defined to be
$ W =  \begin{pmatrix} \mathbb{1} & \sigma^Z & -g \sigma^X \\ & & -J \sigma^Z \\ & & \mathbb{1} \end{pmatrix} $

- Check the energies for the different initial states make sure it matches what you expect.

In [11]:
# the code is in the method `init_H_mpo()`.

In [12]:
psi_Neel = init_Neel_MPS(L=L)
print("energy:", np.sum(psi_Neel.bond_expectation_value(model.H_bonds)))
print("should be", (L-1)* (-J) * (1. * -1.) + L * (-g) * (0.) )

energy: 9.0
should be 9.0


In [13]:
psi_PM = init_PM_MPS(L=L)
print("energy:", np.sum(psi_PM.bond_expectation_value(model.H_bonds)))
print("should be", (L-1)* (-J) * (0. * 0.) + L * (-g) * (1.) )

energy: -11.999999999999993
should be -12.0
